In [ ]:
import numpy as np
import pandas as pd
from math import isnan
import re

In [ ]:
from ape_producer.geom_reader import create_geom_df
from ape_producer.parse_report import make_cov_objects
from ape_producer.sign_conventions import signConventions, square_coordinates

In [ ]:
# ipython magic to autoreload modules for development
%load_ext autoreload
%autoreload 2

In [ ]:
#options
outName = 'Test'
isDT = True
isData = False
is3DOF = False
outname = '_test'
'/eos/cms/store/group/alca_muonalign/'
xmlFileAlign = 'data/run3MC_test_5_01.xml'
reportfile = 'data/run3MC_test_5_01_report.py'
xmlFileIdealGeom = 'data/muonGeometry_IDEAL_AllZeroes.Ape6x6.StdTags.746p3.DBv2.xml'
debug = True

In [ ]:
exec(open(reportfile).read())
aligned_geom_dt, aligned_geom_csc  = create_geom_df(xmlFileAlign)
if not isData:
    ideal_geom_dt, ideal_geom_csc = create_geom_df(xmlFileIdealGeom)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
alignables = ['x','y','z','phix', 'phiy', 'phiz']

for dim in alignables:
    aligned_geom_dt['measured_error_{}'.format(dim)] = aligned_geom_dt[dim] - ideal_geom_dt[dim]
aligned_geom_dt

In [ ]:
def sel_wheel_station(df, wheel, station):
    return df[(df.wheel==wheel) &  (df.station==station)]
def std_dev(x):
    print(x)
    return np.sum(x**2, axis=0)**.5

In [ ]:
error_list = []
wheel_stations = np.unique(aligned_geom_dt[['wheel', 'station']], axis=0)
for wheel, station in wheel_stations:
    error_list.append({"wheel": wheel, "station": station, 
                       **std_dev(sel_wheel_station(aligned_geom_dt,wheel,station)[alignables]).to_dict()
                      })
error_df = pd.DataFrame(error_list)

In [ ]:
error_df

In [ ]:
dt_cov_df, csc_cov_df = make_cov_objects(reports, debug=False)

In [ ]:
hesse_list = []
wheel_stations = np.unique(aligned_geom_dt[['wheel', 'station']], axis=0)
for wheel, station in wheel_stations:
    covMatrix = sel_wheel_station(dt_cov_df,wheel,station).covMatrix.mean()
    hess_stds = np.diagonal(covMatrix)**.5
    hesse_list.append({"wheel": wheel, "station": station, **{dim:hess for hess, dim in zip(hess_stds, alignables)}})
hesse_df = pd.DataFrame(hesse_list)
hesse_df

In [ ]:
sys_df = (error_df**2-hesse_df**2)**.5

In [ ]:
sys_df[['wheel', 'station']] = error_df[['wheel', 'station']]

In [ ]:
sys_df

In [ ]:
def make_heatmap(df, dim):
    twod_edges = (np.linspace(-2.5,2.5,6), np.linspace(.5,4.5,5))
    twod_centers = (np.linspace(-2,2,5), np.linspace(1,4,4))
    heatmap, _, _ = np.histogram2d(df.wheel, df.station, bins=twod_edges, weights=df[dim])
    plt.imshow(heatmap.T)
    plt.colorbar()

In [ ]:
make_heatmap(sys_df, 'x')

In [ ]:
make_heatmap(sys_df, 'y')

In [ ]:
make_heatmap(sys_df, 'z')